In [1]:
from google.colab import files

uploaded = files.upload()


Saving face_mask_detector.keras to face_mask_detector.keras


In [5]:
!pip install streamlit pyngrok


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 65.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 81.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.9 MB/s eta 0:00:00


In [6]:
%%writefile app.py
import streamlit as st
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import image
from PIL import Image

# Load trained model
model = tf.keras.models.load_model("face_mask_detector.keras")

# Function to preprocess the image
def preprocess_image(img):
    img = img.resize((224, 224))  # Resize to match model input size
    img = np.array(img) / 255.0  # Normalize to [0,1]
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    return img

# Streamlit UI
st.title("Face Mask Detection")
st.write("Upload an image, and the model will predict whether a mask is worn.")

# Upload an image
uploaded_file = st.file_uploader("Choose an image...", type=["jpg", "jpeg", "png"])

if uploaded_file is not None:
    image_pil = Image.open(uploaded_file)
    st.image(image_pil, caption="Uploaded Image", use_column_width=True)

    # Preprocess and predict
    img_array = preprocess_image(image_pil)
    prediction = model.predict(img_array)

    # Show results
    result = "No Mask Detected 😷" if prediction[0][0] > 0.5 else "Mask Detected ✅"
    st.subheader(f"Prediction: {result}")


Overwriting app.py


In [7]:
!nohup streamlit run app.py --server.port 8501 --server.address 0.0.0.0 > streamlit.log 2>&1 &


In [9]:
!ngrok authtoken '2tcSJ6lxyRtUeCNRHAHNy2fqZ8l_3rWhYFVh7Nzw4XM5hV5ow'


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [10]:
from pyngrok import ngrok

# Check existing tunnels
active_tunnels = ngrok.get_tunnels()
if active_tunnels:
    public_url = active_tunnels[0].public_url
    print(f"Using existing ngrok tunnel: {public_url}")
else:
    public_url = ngrok.connect(8501).public_url
    print(f"Streamlit app is live at: {public_url}")



Streamlit app is live at: https://01e9-104-199-178-222.ngrok-free.app
